# Adattisztítás és integráció

A valós adathalmazok gyakran zajosak, hiányosak, avagy éppen redundáns információt vagy duplikátum egyedeket tartalmaznak. Ezért a tudásfeltárás folyamatában az adattisztítás és adatintegrálással kezdődik.

Az adattisztítás szerepe javítani az adatok minőségén azáltal, hogy kiszűri és eltávolítja az adatokban fellépő hibákat és inkonzisztenciákat.

Az [adattisztítás](https://hu.wikipedia.org/wiki/Adattiszt%C3%ADtás) során:
"
- felmérjük a hibákat
	- ellenőrizzük az adatfájl szerkezeti épségét
	- a zajt, felesleges információt tartalmazó mezőket javítjuk
	- felmérjük a hiányzó értékeket és amennyiben lehet ezeket pótoljuk
	- felmérjük az adatközlési és adatbeviteli hibákat
		- megvizsgáljuk az egyes változók eloszlását
			- az eloszlások szélein elhelyezkedő extrém értékeket ellenőrizzük
			- felmérjük, hogy az eloszlások megfelelnek-e az előzetes elvárásainknak, vannak-e nem várt sűrűsödések, ritkulások egyes értéktartományokban (például durva kerekítés vagy eltérő mértékegység használata az adatszolgáltatók egy részénél)
		-  megvizsgáljuk, hogy a változók közötti triviális összefüggések teljesülnek-e
-  a hibásnak tűnő adatokat felülvizsgáljuk, javítjuk 
".


# Feladatok

1. Az `egyetemek.txt` fájlból szűrjük ki az államokat és azon belül a városokat, melyben egyetemek találhatóak. Ha vannak duplikátumok, helytelen adatok (pl. számokat tartalmazó államnév), ezeket javítsuk. Vizsgáljuk meg az egyetemek eloszlását államok szerint. Melyik államban van a legtöbb, legkevesebb egyetem?


In [ ]:
with open('/content/sample_data/egyetemek.txt', 'r', encoding='utf-8') as f:
    data = f.readlines()

universities = {}

for line in data:
    line = line.strip()
    if line:
        try:
            state, unis = line.split("[edit]")
            state = state.strip()
            unis = unis.strip().strip("[]")
            uni_list = unis.split(",")
            universities[state] = [uni.strip() for uni in uni_list]
        except ValueError:
            print(f"Could not process line: {line}")

for line in data:
    line = line.strip()
    if line:
        state, unis = line.split("[edit]") if "[edit]" in line else (line, "")
        state = state.strip()
        unis = unis.strip().strip("[]")
        uni_list = unis.split(",")
        universities[state] = [uni.strip() for uni in uni_list]

counts = {state: len(unis) for state, unis in universities.items()}

most_unis_state = max(counts, key=counts.get)
least_unis_state = min(counts, key=counts.get)

print("**********************************************************")
print(f"Most Uni: {most_unis_state} ({counts[most_unis_state]} db)")
print(f"Least Uni: {least_unis_state} ({counts[least_unis_state]} db)")
print("**********************************************************")

# print(universities)

2. Bővítsük ki az adatbázisunkat egy oszloppal, mely tartalmazza az államok rövidítését is (pl. Texas - TX, California - CA stb.).  [Forrás](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations).

In [ ]:
df_codes = pd.read_csv('/content/content/sample_data/shortNames.csv',names = ['State', 'Code'])
df = df.merge(df_codes, on = 'State', how='left')

In [ ]:
d = df[['State','Code']].copy()
d['szam'] = 1
uniabs = d.groupby(['State']).sum()
uniabs = uniabs.merge(df_codes, on = 'State', how='left')
uniabs

3. Az adatbázist integráljuk a [List of states and territories of the United States by population](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population) linken szereplő népszámlálási adatokkal és számoljuk ki államonként hány főre jut egy egyetemi város.

In [ ]:
df_pop = pd.read_csv('/content/sample_data/popular.csv',names = ['State', 'Popular'])

In [ ]:
d = df[['State','Code']].copy()
d['szam'] = 1
by_state = d.groupby(['State'])
by_state = by_state.sum()
new_pop = by_state.merge(df_pop, on = 'State', how='left')
new_pop = new_pop.merge(df_codes, on = 'State', how='left')
new_pop['fo/egyet'] = new_pop['Popular']/new_pop['szam']
new_pop.head()


4. Hasonlóan, a [List of U.S. states and territories by area](https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area) linken szereplő területi adatok integrálásával, számoljuk ki államonként átlagban hány négyzetkilométerre jut egy egyetemi város.

In [5]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Betöltjük az area táblázatot a wikipédiáról
url = "https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area"
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")
table = soup.find_all("table")[0]
df_area = pd.read_html(str(table))[0]
df_area.columns = df_area.columns.droplevel(0)

# Betöltjük az egyetemek fájlt és számoljuk, hány egyetem van városonként
with open("/content/sample_data/egyetemek.txt", "r") as f:
    lines = f.readlines()
city_university_counts = {}
for line in lines:
    city = line.split(" (")[0]
    count = line.count(",") + 1
    city_university_counts[city] = count

# Összefésüljük az államokat és a városokat
df_cities = pd.DataFrame.from_dict(city_university_counts, orient="index", columns=["university_count"])
df_cities.index.name = "City"
df_cities = df_cities.reset_index()
df_cities["State"] = df_cities["City"].apply(lambda x: x.split(", ")[-1])
df_cities["City"] = df_cities["City"].apply(lambda x: x.split(", ")[0])

# print(df_area.columns)

df_area = df_area.loc[:, ~df_area.columns.duplicated()]
# print(df_area.columns)

# Számoljuk ki az egyetemi városok összes területét az államonkénti összes területből kivonva az államon kívüli területeket
df_area = df_area[df_area["Rank"] != "—"]
df_area["State"] = df_area["State"].str.replace("\[.*\]", "", regex=True)
df_area["sq mi"] = df_area["sq mi"].astype(str)
df_area["sq mi"] = df_area["sq mi"].str.replace(",", "", regex=True).astype(float)
df_area = df_area.groupby("State").agg({"sq mi": sum})
df_area = df_area.merge(df_cities, on="State", how="outer")
df_area["University area per state"] = df_area["sq mi"] / df_area["university_count"]

# Kiírjuk az eredményt
print(df_area[["State", "University area per state"]])

                                                 State  \
0                                 48 contiguous states   
1                   50 states and District of Columbia   
2                                              Alabama   
3                                               Alaska   
4    All 50 states, District of Columbia, and U.S. ...   
..                                                 ...   
581                                         Whitewater   
582                                    Wyoming[edit]\n   
583                                            Laramie   
584                                     123456[edit]\n   
585                                               +000   

     University area per state  
0                          NaN  
1                          NaN  
2                          NaN  
3                          NaN  
4                          NaN  
..                         ...  
581                        NaN  
582                        NaN  
583   

5. A [példa](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/), a kivitelt, exportmennyiséget ábrázolja az Amerikai Egyesült Államok térképen, államonként lebontva ezt. Készítsünk hasonló ábrákat az egyetemek abszolút, lakoság és terület szerinti eloszlásáról is. Az ábrákat exportáljuk kép formájában.
![](https://i.ibb.co/s1zdnLY/mapplot.png)

Ábra 1. [Térképen való ábrázolás](https://plotly.com/python/maps/) Plotly segítségével. [Forrás](https://datasciencechalktalk.com/2019/09/28/analyzing-u-s-exports-with-plotly/).

In [6]:
with open('/content/sample_data/egyetemek.txt', 'r') as file:
    lines = file.readlines()

universities = {}
for line in lines:
    # ellenőrizzük, hogy az adott sor egy államot tartalmaz-e
    if '[edit]' in line:
        state = line.replace('[edit]\n', '')
        universities[state] = {}
    else:
        # a sorban szereplő városok és egyetemek feldolgozása
        city = line.split(' (')[0]
        universities[state][city] = line.count(',') + 1

# az egyetemek száma államonként
state_universities = {}
for state, cities in universities.items():
    state_universities[state] = sum(cities.values())

# a legtöbb egyetemmel rendelkező városok és államok meghatározása
most_universities_city = max((city, count) for state in universities for city, count in universities[state].items())
most_universities_state = max(state_universities.items(), key=lambda x: x[1])

print("Egyetemek száma államonként:\n", state_universities)
print("Egyetemek száma városonként és államonként:\n", universities)
print("A legtöbb egyetemmel rendelkező város: {} ({})".format(most_universities_city[0], most_universities_city[1]))
print("A legtöbb egyetemmel rendelkező állam: {} ({})".format(most_universities_state[0], most_universities_state[1]))


Egyetemek száma államonként:
 {'Alabama': 10, 'Alaska': 1, 'Arizona': 3, 'Arkansas': 11, 'California': 57, 'Colorado': 10, 'Connecticut': 14, 'Delaware': 2, 'Florida': 18, 'Georgia': 21, 'Hawaii': 1, 'Idaho': 3, 'Illinois': 10, 'Indiana': 14, 'Iowa': 15, 'Kansas': 8, 'Kentucky': 13, 'Louisiana': 9, 'Maine': 11, 'Maryland': 13, 'Massachusetts': 47, 'Michigan': 23, 'Minnesota': 27, 'Mississippi': 5, 'Missouri': 12, 'Montana': 3, 'Nebraska': 7, 'Nevada': 4, 'New Hampshire': 8, 'New Jersey': 16, 'New Mexico': 5, 'New York': 65, 'North Carolina': 27, 'North Dakota': 2, 'Ohio': 23, 'Oklahoma': 12, 'Oregon': 13, 'Pennsylvania': 59, 'Rhode Island': 9, 'South Carolina': 21, 'South Dakota': 4, 'Tennessee': 28, 'Texas': 32, 'Utah': 6, 'Vermont': 7, 'Virginia': 25, 'Washington': 6, 'West Virginia': 9, 'Wisconsin': 16, 'Wyoming': 1, '123456': 1}
Egyetemek száma városonként és államonként:
 {'Alabama': {'Auburn': 1, 'Florence': 1, 'Jacksonville': 1, 'Livingston': 1, 'Montevallo': 1, 'Troy': 1, 'Tusc

6. Számoljuk ki, hány egyetem van városonként, majd összesítve államonként. Ehhez az `egyetemek.txt` fájlban miután megkapunk egy várost, az utána következő kerek zárójelek között megszámoljuk, hány vesszővel elválasztott karakterlánc található. 
Pl. a következő sorban:	`Claremont (Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University)[5]` -> 6 vessző van a kerek zárójelek között, tehát 7 egyetem van a városban.    
Melyik államban és melyik városban van a legtöbb egyetem?

In [ ]:
def clean_town(item):
  return item[:item.find('(') - 1]
def clean_state(item):
  return item[:item.find('[')]
def what_univers(item):
  return len(item[item.find('(') - 1:].split(','))

df1['NumberUniv'] = df1.Town.apply(what_univers)
df1['Town'] = df1.Town.apply(clean_town)
df1['State'] = df1.State.apply(clean_state)
df1

In [ ]:
by_state = df1.groupby(['State'])
maxi = by_state.NumberUniv.sum().max()
mini = by_state.NumberUniv.sum().min()
print(maxi,mini)

by_town = df1.groupby(['Town'])
maxi = by_town.NumberUniv.sum().max()
mini = by_town.NumberUniv.sum().min()
print(maxi,mini)